**Use of Grid Search with Random Forest on the Boston Housing data**

**AWS Configuration required**

Runremotely use your credentials stored in ~/.aws/credential

Make sure you have install aws-cli and it's configured

https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))
from runremotely.runremotely import runremotely

In [3]:
# Import the usual suspects
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

**Load and Split data**

In [4]:
data = pd.read_csv('data.csv')

y = np.log(data.SalePrice)
X = data.drop(['SalePrice', 'Id'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(
                          X, y, random_state=42, test_size=.33)

print(X_train.shape)

(978, 246)


**Initialize the Random Forest and the Grid Search**

In [5]:
# Initialize Random Forest
rf = RandomForestRegressor()

In [6]:
# Choose parameters for Grid Search
rf_params = {'n_estimators': range(150, 300, 10), 'max_depth': range(3, 15, 1),'min_samples_leaf':[0.04,0.06,0.08], 'max_features':[0.2,0.4,0.6,0.8]}

In [12]:
# Don't forget the n_jobs parameters on a multi-cpu machine!
grid = GridSearchCV(estimator = rf,
                         param_grid = rf_params,
                         cv=10,
                         scoring='neg_mean_squared_error',
                         verbose=1,
                         n_jobs=-1)

In [17]:
@runremotely(globals(), instance="c5.9xlarge")
def execute():
    return grid.fit(X_train, y_train)

In [18]:
results = execute()

Requirements files: /var/folders/ms/xwkpspgd3kn4y2cb_q4639zh0000gn/T/tmpqbig1o_w.reqs
Creating key pair [/var/folders/ms/xwkpspgd3kn4y2cb_q4639zh0000gn/T/tmpf4w53t7s.tp]
Launching instance (c5.9xlarge)
EC2 instance created:
	Id: i-0510141482d4b4ca0
	Public IP address: 54.171.72.36
Transfering [/var/folders/ms/xwkpspgd3kn4y2cb_q4639zh0000gn/T/tmpf05umr2h.tp] to ec2
Transfering [../template.py] to ec2
Transfering [/var/folders/ms/xwkpspgd3kn4y2cb_q4639zh0000gn/T/tmpqbig1o_w.reqs] to ec2
Executing [sudo yum -y install python37]
Executing [curl -O https://bootstrap.pypa.io/get-pip.py]
Error:   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1824k  100 1824k    0     0  33.6M      0 --:--:-- --:--:-- --:--:-- 33.6M

Executing [sudo  python3 get-pip.py --user]
Error:   WARNING: The scripts pip, pip3 and pip3.7 are installed in '/root/.local/bin' which is not on PATH.
  Consider add

Retrieving [result.pickle] from ec2



/Users/odyssee/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.23.0 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/odyssee/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.23.0 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/odyssee/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator GridSearchCV from version 0.23.0 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [19]:
results.best_score_

-0.032043335051350805

In [20]:
results.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=14, max_features=0.6, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=0.04,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=230, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)